# Density matrix embedding theory (DMET)

## Table of contents:
* [1. Introduction](#1)
* [2. Theory of DMET](#2)
* [3. First example: DMET-CCSD on Butane](#3)
* [4. Second example: DMET-CCSD on an hydrogen ring](#4)
>* [4.1 Why not only using VQE?](#41)
>* [4.2 Section](#42)

Before the introduction, let's do all imports needed in this notebook.

In [16]:
# Import for a pretty jupyter notebook...
import json

# The minimal import for DMET.
from qsdk.problem_decomposition.dmet.dmet_problem_decomposition import DMETProblemDecomposition
# Ability to change localization method.
from qsdk.problem_decomposition.dmet.dmet_problem_decomposition import Localization
# Use for VQE ressources estimation vs DMET.
from qsdk.electronic_structure_solvers.vqe_solver import VQESolver

## 1. Introduction <a class="anchor" id="1"></a>
One of the main objectives of quantum chemistry calculations in the area of materials science is to solve the electronic structure problem, $H\Psi=E\Psi$, as accurately as possible, in order to accelerate the materials design process. In the first example, the butane molecule is shown as an example. 

<img src="img/exact.png" alt="exact" width="200" />

The computational cost for performing accurate calculations of the electronic structure of molecules, however, is usually very expensive. For example, the cost of performing the full CI calculation scales exponentially on a classical computer as the size of the system increases. Therefore, when we target large-sized molecules, those relevant for industry problems, it becomes essential to employ an appropriate strategy for reducing the computational cost. The difficulty is to employ a strategy that consolidate accuracy while reducing computational costs when performing electronic structure calculations. Next, we will developp one of such strategy called Density matrix embedding theory (DMET).

## 2. Theory of DMET <a class="anchor" id="2"></a>

The idea is to decompose a molecular system into its constituent fragments and its environment. Each fragment are treated independently and recombined at the end to recover the full molecular energy. This has the advantages of cutting down the maximal computational complexity depending on the biggest fragment and being naturally implemented on parrallel computer. On the other side, the fragmenting comes at the cost of reducing accuraccy of the computation.

First, the environment is calculated using a less-accurate method than will be used to calculate the electronic structure of a fragment. Then, the electronic structure problem for a given fragment is solved to a high degree of accuracy, which includes the quantum mechanical effects of the environment. The quantum mechanical description is updated (i.e., solved iteratively as shown below) by incorporating the just-performed highly accurate calculation. In the following schematic illustration, the molecule shown above is decomposed into fragments. Each molecular fragment CH$_\mathrm{3}$ and CH$_\text{2}$ are the fragments chosen for the electronic structure calculation, with the rest of the molecular system being the surrounding environment.

<img src="img/iterations.png" alt="iterations" width="600" />

One successful decomposition approach is the DMET method. The DMET method decomposes a molecule into fragments, and each fragment is treated as an open quantum system that is entangled with each of the other fragments, all taken together to be that fragment's surrounding environment (or "bath"). In this framework, the electronic structure of a given fragment is obtained by solving the following Hamiltonian, by using a highly accurate quantum chemistry method, such as the full CI method or a coupled-cluster method.

$$ H_{I}=\sum^{\text{frag}+\text{bath}}_{rs}  \left[ h_{rs} + \sum_{mn} \left[ (rs|mn) - (rn|ms) \right] D^{\text{(mf)env}}_{mn} \right] a_{r}^{\dagger}a_{s} + \sum_{pqrs}^{\text{frag}+\text{bath}} (pq|rs) a_{p}^{\dagger}a_{r}^{\dagger}a_{s}a_{q} - \mu\sum_{r}^{\text{frag}} a_{r}^{\dagger}a_{r} $$

The expression $\sum_{mn} \left[ (rs|mn) - (rn|ms) \right] D^{\text{(mf)env}}_{mn}$ describes the quantum mechanical effects of the environment on the fragment, where $D^{\text{(mf)env}}_{mn}$ is the mean-field electronic density obtained by solving the Hartree&ndash;Fock equation. The quantum mechanical effects from the environment are incorporated through the one-particle term of the Hamiltonian. The extra term $\mu\sum_{r}^{\text{frag}} a_{r}^{\dagger}a_{r}$ ensures, through the adjustment of $\mu$, that the number of electrons in all of the fragments, taken together, becomes equal to the total number of electrons in the entire system.

## 3. First example: DMET-CCSD on Butane <a class="anchor" id="3"></a>

The first example will show how to partition the system into fragments. A different method for solving electronic structures can be chosen for each fragments, but here we will stick with CCSD. The first thing to do is building the butane PySCF molecule object.

In [42]:
butane = """
C   2.142   1.395  -8.932
H   1.604   0.760  -8.260              
H   1.745   2.388  -8.880
H   2.043   1.024  -9.930
C   3.631   1.416  -8.537
H   4.169   2.051  -9.210
H   3.731   1.788  -7.539             
C   4.203  -0.012  -8.612
H   3.665  -0.647  -7.940
H   4.104  -0.384  -9.610
C   5.691   0.009  -8.218  
H   6.088  -0.983  -8.270
H   5.791   0.381  -7.220
H   6.230   0.644  -8.890
"""

# Building the PySCF object.
mol = gto.Mole()
mol.atom = butane
mol.basis = "minao"
mol.charge = 0
mol.spin = 0
mol.build()

The options for the DMET decomposition method are stored in a python dictionary.
* **molecule**: The PySCF molecule object.
* **fragment_atoms**: List for the number of atoms in each fragment. Each atoms are chosen with their order in the coordinates definition (variable `butane`). Also, the sum of all number must be equal to total number of atom in the molecule.
* **fragment_solvers**: A tring or a list of string representing the solver for each fragment. The number of items in the list must be equal to the number of fragment. There is one exception that is when a single string is defined. At this time, the solver is the same for all fragments.
* **verbose**: Activate verbose output.

The next step `dmet.build()` ensures that the fragments and envrionement orbitals are defined according to the localization scheme.

In [39]:
dmet_options = {"molecule": mol,
               "fragment_atoms": [4, 3, 3, 4],
               "fragment_solvers": "ccsd",
               "verbose": True
               }

dmet = DMETProblemDecomposition(dmet_options)
dmet.build()

In [40]:
dmet_energy = dmet.simulate()

 	Iteration =  1
 	----------------
 
		Fragment Number : #  1
		------------------------
		Fragment Energy                 =    -72.0540582760
		Number of Electrons in Fragment =     16.0000000000

		Fragment Number : #  2
		------------------------
		Fragment Energy                 =    -72.4095411539
		Number of Electrons in Fragment =     14.0000000000

		Fragment Number : #  3
		------------------------
		Fragment Energy                 =    -72.4172977137
		Number of Electrons in Fragment =     14.0000000000

		Fragment Number : #  4
		------------------------
		Fragment Energy                 =    -72.0631745662
		Number of Electrons in Fragment =     16.0000000000

 	Iteration =  2
 	----------------
 
		Fragment Number : #  1
		------------------------
		Fragment Energy                 =    -72.0544828916
		Number of Electrons in Fragment =     16.0000000000

		Fragment Number : #  2
		------------------------
		Fragment Energy                 =    -72.4103731263
		Number of E

## 4. Second example: DMET-CCSD on an hydrogen ring <a class="anchor" id="4"></a>

### 4.1 Why not only using VQE? <a class="anchor" id="41"></a>

In [31]:
H10="""
H          1.6180339887          0.0000000000          0.0000000000
H          1.3090169944          0.9510565163          0.0000000000
H          0.5000000000          1.5388417686          0.0000000000
H         -0.5000000000          1.5388417686          0.0000000000
H         -1.3090169944          0.9510565163          0.0000000000
H         -1.6180339887          0.0000000000          0.0000000000
H         -1.3090169944         -0.9510565163          0.0000000000
H         -0.5000000000         -1.5388417686          0.0000000000
H          0.5000000000         -1.5388417686          0.0000000000
H          1.3090169944         -0.9510565163          0.0000000000
"""

mol = gto.Mole()
mol.atom = H10
mol.basis = "minao"
mol.charge = 0
mol.spin = 0
mol.build()

In [32]:
vqe_options = {"molecule": mol,
               "qubit_mapping": "jw",
               "verbose": False
               }

In [33]:
vqe = VQESolver(vqe_options)

In [34]:
vqe.build()

In [36]:
vqe_resources = vqe.get_resources()
print(json.dumps(vqe_resources, indent=2))

{
  "qubit_hamiltonian_terms": 4745,
  "circuit_width": 20,
  "circuit_gates": 57126,
  "circuit_2qubit_gates": 37248,
  "circuit_var_gates": 2252,
  "vqe_variational_parameters": 350
}


### 4.2 Section <a class="anchor" id="42"></a>

Here, we demonstrate how to perform  DMET calculations using 1QBit's OpenQEMIST software package. Harnessing the power of emerging quantum computing technologies combined with state-of-the-art classical techniques, OpenQEMIST is able to deliver either state-of-the-art classical solutions or, with the flip of a switch, map a computationally challenging problem onto quantum computing architectures without the need for any additional programming, neither by a user nor a developer.

We have selected a ring of 10 hydrogen atoms as a simple example of a molecular system. The distance between adjacent hydrogen atoms has been set to 1$~$Å.

In [28]:
dmet_options = {"molecule": mol,
               "fragment_atoms": [1]*10,
               "fragment_solvers": "vqe",
               "verbose": False
               }

dmet = DMETProblemDecomposition(dmet_options)
dmet.build()

In [29]:
dmet_resources = dmet.get_resources()
print(json.dumps(dmet_resources[0], indent=2))

{
  "qubit_hamiltonian_terms": 27,
  "circuit_width": 4,
  "circuit_gates": 158,
  "circuit_2qubit_gates": 64,
  "circuit_var_gates": 12,
  "vqe_variational_parameters": 2
}


In [ ]:
dmet_energy = dmet.simulate()

In [14]:
#from openqemist.electronic_structure_solvers import FCISolver
#from openqemist.electronic_structure_solvers import CCSDSolver

The results obtained from the DMET-CCSD method (using problem decomposition) are almost identical to those of the Full CI method (without using problem decomposition). When we decompose the ring of atoms into fragments, one of which includes only one hydrogen atom, the DMET method creates a fragment orbital (left: the single orbital distribution is shown in both pink and blue, with the colours depicting the phases) and the bath orbital (right: the single orbital distribution of the remaining nine hydrogen atoms is shown in both pink and blue, with the colours depicting the phases). 

<img src="img/frag_and_bath.png" alt="fragment_and_bath_orbitals" width="450"/>

Then, the DMET Hamiltonian will consist of only two electrons and two (i.e., fragment and bath) orbitals. Therefore, the CCSD solver, which treats single- and double-excitations, will provide results equivalent to those of the Full CI solver. This is why the results of the DMET-CCSD and Full CI methods almost coincide.

The DMET calculations require only about 0.6% of the amplitudes compared to the CCSD calculation for the full system consisting of 10 hydrogen atoms. The number of Hamiltonian terms in the DMET calculation in the spacial orbital basis is only 20% (17% in the spin orbital basis) of the full system. A large amount of computational resources are saved while maintaining the same level of accuracy in the calculations.